In [1]:
!pip install update pandas

In [2]:
!pip install scikit-learn konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 30.9 MB/s eta 0:00:00


In [5]:
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Okt


In [6]:
from google.colab import files
import os

data_dir = 'data'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)
files.upload()
os.chdir('..')

Saving news.csv to news.csv


In [7]:
okt = Okt()
df = pd.read_csv('./data/news.csv')


print(f"Loaded {len(df)} news articles.")

Loaded 1600 news articles.


In [8]:
# 한국어 전처리 함수
def preprocess_korean(text):
    if pd.isna(text):
        return ""
    # 기본: 특수문자, 숫자, 영어 제거 (형태소 분석 없이)
    text = re.sub(r'[^가-힣\s]', ' ', text)  # 한글과 공백만 유지
    text = re.sub(r'\s+', ' ', text).strip()  # 다중 공백 제거

    # 형태소 분석: 명사 추출
    tokens = okt.nouns(text)
    # 불용어 제거 (간단한 한국어 불용어 리스트)
    stopwords_kr = {'것', '수', '이', '가', '를', '을', '에', '의', '와', '과', '로', '으로', '에서', '이다', '되다', '있다', '하다', '않다'}
    tokens = [word for word in tokens if len(word) > 1 and word not in stopwords_kr]
    return ' '.join(tokens)

# 데이터 전처리
df['processed_news'] = df['news'].apply(preprocess_korean)
processed_data = df['processed_news'] #.tolist()

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 사용하여 BoW 생성
count_vectorizer = CountVectorizer(max_features=1000, min_df=2, max_df=0.8)
bow_matrix = count_vectorizer.fit_transform(processed_data)

print("BoW 행렬 형태:", bow_matrix.shape)

BoW 행렬 형태: (1600, 1000)


In [10]:
# 단어별 총 빈도수 계산
word_counts = bow_matrix.sum(axis=0)

# 단어와 빈도수를 딕셔너리로 매핑
word_freq = dict(zip(count_vectorizer.get_feature_names_out(), word_counts.tolist()[0]))



In [11]:
# 빈도수에 따라 단어 정렬
sorted_word_freq = sorted(word_freq.items(), key=lambda item: item[1], reverse=True)

# 상위 30개 단어 출력
print("상위 30개 단어와 빈도수:")
for word, freq in sorted_word_freq[:30]:
    print(f"{word}: {freq}")

상위 30개 단어와 빈도수:
기자: 1276
한국: 1161
서울: 1063
지난: 973
북한: 836
시간: 828
대해: 809
미국: 803
대한: 792
대통령: 734
이번: 708
통해: 699
때문: 687
위해: 685
관련: 684
중국: 670
사람: 663
경찰: 653
이후: 624
정부: 615
대표: 582
문제: 571
사실: 561
조사: 561
경우: 547
지난해: 542
후보: 536
관계자: 531
대회: 524
우리: 514
